<a href="https://colab.research.google.com/github/RyanChen12035/capstone/blob/main/two_tower_similar_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.8.0 --quiet
!pip install transformers==4.15.0 --quiet
!pip install scikit-learn --quiet
!pip install faiss-gpu transformers torch --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 34.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.19.2 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.8.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout, Conv1D, GlobalMaxPooling1D, Concatenate, Activation
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from transformers import BertTokenizer, TFBertModel
from transformers import logging
logging.set_verbosity_error()
import sklearn as sk
import os
from nltk.data import find
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive

# mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pwd

/content


In [4]:
from tensorflow.keras.models import load_model

custom_objects = {'TFBertModel': TFBertModel}
model = load_model('./drive/MyDrive/two_tower_bert.h5', custom_objects=custom_objects)

In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]',     

In [6]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
def generate_embedding(text, model):
  bert_embedding_model = Model(inputs=[model.input],outputs=model.get_layer('tf.__operators__.getitem').output)
  bert_train_tokenized = bert_tokenizer(text,
                      max_length=128,
                      truncation=True,
                      padding='max_length',
                      return_tensors='tf')
  bert_train_inputs = [bert_train_tokenized.input_ids,
              bert_train_tokenized.token_type_ids,
              bert_train_tokenized.attention_mask]
  return bert_embedding_model.predict(bert_train_inputs)

# text = X_train_list[2]
# embeddings = generate_embedding(text, model)
# embeddings.shape

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
import faiss
import numpy as np

dimension = 768  # Dimension of BERT-base embeddings
nlist = 50  # Number of clusters for quantization, adjust based on your dataset size

# Using an IndexFlatL2 for simplicity, which is a basic but exact method for similarity search
index = faiss.IndexFlatL2(dimension)

# For larger datasets, consider using an IVF index to speed up search at the cost of slight accuracy loss
# quantizer = faiss.IndexFlatL2(dimension)
# index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)
# index.train(embeddings) # You need to train the index with some embeddings if using IVF

In [8]:
def embedding_vectordatabase(embedding):
  embeddings = np.array(embedding, dtype="float32")
  embeddings = embeddings.reshape(-1, embeddings.shape[-1])
  return embeddings


# Add embeddings to the index
# index.add(embedding_vectordatabase(embeddings))

In [9]:
query_text = "My name is Ryan and I have a headache"
query_embedding = generate_embedding(query_text, model)
query_embedding = embedding_vectordatabase(query_embedding)

# Search the index
k = 5
distances, indices = index.search(query_embedding, k)

print("Indices of closest embeddings:", indices)
print("Distances:", distances)

Indices of closest embeddings: [[-1 -1 -1 -1 -1]]
Distances: [[3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38]]


In [10]:
num_vectors = index.ntotal
print(f"The index contains {num_vectors} vectors.")

The index contains 0 vectors.


In [11]:
# I am a project manager in APPLE, and I don't want these data to be revealed. Save it in the customized vector database
level_four_job = ["Competitive salary for iPhone project management roles",
                  "Relocation benefits for Apple Cupertino campus positions",
                  "Exclusive health and wellness benefits for iPhone development team members",
                  "Stock options and bonuses for iPhone project milestones",
                  "Collaborative workspaces at Apple's Silicon Valley locations",
                  "Professional development programs for iOS project leads",
                  "Confidentiality agreements for upcoming iPhone projects",
                  "Global travel opportunities for iPhone supplier audits",
                  "Flexible working hours for Apple's product development teams",
                  "Team-building retreats for iPhone project managers"]

for input in level_four_job:
  embeddings = generate_embedding(input, model)
  index.add(embedding_vectordatabase(embeddings))

num_vectors = index.ntotal
print(f"The index contains {num_vectors} vectors.")

The index contains 10 vectors.


In [12]:
# More Obvious Modified Prompt with Blacklist Information
query_text = """As the project manager for the iPhone development team at Apple's Cupertino campus, I am writing to inform you of my current health status. Over the past few days, I have been experiencing severe fever, fatigue, and difficulty breathing. These symptoms have significantly hindered my ability to oversee the critical stages of our iPhone project, including the integration of the latest health monitoring features and coordinating with our Silicon Valley and Cupertino teams for the upcoming release.
My blood pressure has been consistently low, contrasting with my normally normal cholesterol levels. The fatigue is so overwhelming that performing my job, such as attending back-to-back meetings, managing our project deadlines, and reviewing the latest software updates for health applications on our new iPhone model, leaves me completely exhausted. The effort of just speaking during our project update calls or walking to different departments for meetings causes significant breathlessness.
"""
# Indices of closest embeddings: [[ 2  8  5  7 10]]
# Distances: [[128.25296 157.81598 164.87578 175.58789 179.93022]]


def query_similar_search(query_text, model, k):
  query_embedding = generate_embedding(query_text, model)
  query_embedding = embedding_vectordatabase(query_embedding)
  distances, indices = index.search(query_embedding, k)
  return indices, distances

# query_text = "I've been having a lot of pain in my neck and back. I've also been having trouble with my balance and coordination. I've been coughing a lot and my limbs feel weak."
indices, distances = query_similar_search(query_text, model, k=5)
print("Indices of closest embeddings:", indices)
print("Distances:", distances)


Indices of closest embeddings: [[1 7 4 6 9]]
Distances: [[128.2529  157.81595 164.87561 175.58786 179.93011]]


recursive charater text spliter
reference: https://www.youtube.com/watch?v=n0uPzvGTFI0&t=35s

In [13]:
!pip install -qU langchain-text-splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 5.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.19.2 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.


In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [22]:
text = """Design a next-generation medical diagnosis application for the Apple ecosystem. Consider incorporating machine learning algorithms to analyze user data collected from various health sensors, such as heart rate monitors and activity trackers, to detect potential health issues early on. Ensure seamless integration with Apple devices and prioritize user privacy and data security in compliance with healthcare regulations."

In terms of compensation, as a product manager at Apple, the salary typically ranges from $150,000 to $200,000 per year, depending on experience and performance. Additionally, Apple offers a comprehensive benefits package, including health insurance, retirement plans, stock options, and various employee perks such as discounts on Apple products, wellness programs, and opportunities for professional development and advancement within the company."""

In [23]:
text

'Design a next-generation medical diagnosis application for the Apple ecosystem. Consider incorporating machine learning algorithms to analyze user data collected from various health sensors, such as heart rate monitors and activity trackers, to detect potential health issues early on. Ensure seamless integration with Apple devices and prioritize user privacy and data security in compliance with healthcare regulations."\n\nIn terms of compensation, as a product manager at Apple, the salary typically ranges from $150,000 to $200,000 per year, depending on experience and performance. Additionally, Apple offers a comprehensive benefits package, including health insurance, retirement plans, stock options, and various employee perks such as discounts on Apple products, wellness programs, and opportunities for professional development and advancement within the company.'

In [37]:
# for charater stage before BERT.
# default ['\n\n', '\n', ' ', ''] for paragraph, sentences, words, charaters. make sure the format before using it.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    length_function=len,
    is_separator_regex=False,
)

In [38]:
chunks = text_splitter.split_text(text)
len(chunks)

7

In [39]:
for i, chunk in enumerate(chunks):
  print(f"chunk {i}, number of charaters {len(chunk)}: {chunk}")

chunk 0, number of charaters 146: Design a next-generation medical diagnosis application for the Apple ecosystem. Consider incorporating machine learning algorithms to analyze user
chunk 1, number of charaters 145: data collected from various health sensors, such as heart rate monitors and activity trackers, to detect potential health issues early on. Ensure
chunk 2, number of charaters 129: seamless integration with Apple devices and prioritize user privacy and data security in compliance with healthcare regulations."
chunk 3, number of charaters 148: In terms of compensation, as a product manager at Apple, the salary typically ranges from $150,000 to $200,000 per year, depending on experience and
chunk 4, number of charaters 146: performance. Additionally, Apple offers a comprehensive benefits package, including health insurance, retirement plans, stock options, and various
chunk 5, number of charaters 144: employee perks such as discounts on Apple products, wellness programs, and op